# Clustering Crypto

In [4]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [6]:
# Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col='Unnamed: 0')
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [7]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [8]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns=["IsTrading"], inplace=True)

In [9]:
# Remove rows that have at least 1 null value.
crypto_df.dropna()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [11]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0.0]
crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [12]:
# Create a new DataFrame that holds only the cryptocurrencies names.
# Convert CoinName column to a list
crypto_names_df = crypto_df[['CoinName']]
crypto_names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [13]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=["CoinName"], inplace=True)
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [14]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
X.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(X)
print(crypto_scaled[0:7])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [16]:
# Using PCA to reduce dimension to three principal components.
# Initialize PCA model
pca = PCA(n_components=3)

# Get three principal components for the crypto data
crypto_pca = pca.fit_transform(crypto_scaled)
print(crypto_pca[0:5])

[[-0.33227405  1.05318635 -0.56543872]
 [-0.31562531  1.05332603 -0.56588881]
 [ 2.31138485  1.59079721 -0.6448151 ]
 [-0.14065185 -1.337178    0.20171979]
 [-0.16231462 -1.99872878  0.36666094]]


In [17]:
# Create a DataFrame with the three principal components.
# Transform PCA data to a dataframe
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=X.index.copy()
)

pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.332274,1.053186,-0.565439
404,-0.315625,1.053326,-0.565889
1337,2.311385,1.590797,-0.644815
BTC,-0.140652,-1.337178,0.201720
ETH,-0.162315,-1.998729,0.366661
LTC,-0.166654,-1.090091,-0.004774
DASH,-0.387538,1.207958,-0.513974
XMR,-0.160868,-2.243226,0.402129
ETC,-0.160759,-1.998831,0.366637
ZEC,-0.159672,-2.097812,0.327086


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [18]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

In [19]:
# Plot the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [70]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions[0:20])

[0 0 0 2 2 2 0 2 2 2 0 2 0 0 2 0 2 2 0 0]


In [71]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# clustered_df = pd.concat([crypto_df, pcs_df])
# clustered_df.head()

clustered_df = crypto_df.join(pcs_df)
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.332274,1.053186,-0.565439
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.315625,1.053326,-0.565889
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.311385,1.590797,-0.644815
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.140652,-1.337178,0.201720
ETH,Ethash,PoW,1.076842e+08,0,-0.162315,-1.998729,0.366661
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.166654,-1.090091,-0.004774
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.387538,1.207958,-0.513974
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.160868,-2.243226,0.402129
ETC,Ethash,PoW,1.133597e+08,210000000,-0.160759,-1.998831,0.366637
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.159672,-2.097812,0.327086


In [72]:
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = crypto_names_df['CoinName']
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.332274,1.053186,-0.565439,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.315625,1.053326,-0.565889,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.311385,1.590797,-0.644815,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.140652,-1.337178,0.201720,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.162315,-1.998729,0.366661,Ethereum


In [73]:
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.332274,1.053186,-0.565439,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.315625,1.053326,-0.565889,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.311385,1.590797,-0.644815,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.140652,-1.337178,0.201720,Bitcoin,2
ETH,Ethash,PoW,1.076842e+08,0,-0.162315,-1.998729,0.366661,Ethereum,2
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.166654,-1.090091,-0.004774,Litecoin,2
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.387538,1.207958,-0.513974,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.160868,-2.243226,0.402129,Monero,2
ETC,Ethash,PoW,1.133597e+08,210000000,-0.160759,-1.998831,0.366637,Ethereum Classic,2
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.159672,-2.097812,0.327086,ZCash,2


In [74]:
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.332274,1.053186,-0.565439,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.315625,1.053326,-0.565889,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.311385,1.590797,-0.644815,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.140652,-1.337178,0.201720,Bitcoin,2
ETH,Ethash,PoW,1.076842e+08,0,-0.162315,-1.998729,0.366661,Ethereum,2
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.166654,-1.090091,-0.004774,Litecoin,2
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.387538,1.207958,-0.513974,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.160868,-2.243226,0.402129,Monero,2
ETC,Ethash,PoW,1.133597e+08,210000000,-0.160759,-1.998831,0.366637,Ethereum Classic,2
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.159672,-2.097812,0.327086,ZCash,2


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [75]:
# Convert list of algorithm names to a Pandas Series
algor_array = clustered_df["Algorithm"].to_numpy()
print(algor_array)

['Scrypt' 'Scrypt' 'X13' 'SHA-256' 'Ethash' 'Scrypt' 'X11'
 'CryptoNight-V7' 'Ethash' 'Equihash' 'SHA-512' 'Multiple' 'SHA-256'
 'SHA-256' 'Scrypt' 'X15' 'X11' 'Scrypt' 'Scrypt' 'Scrypt' 'Multiple'
 'Scrypt' 'SHA-256' 'Scrypt' 'Scrypt' 'Scrypt' 'Quark' 'Groestl' 'Scrypt'
 'Scrypt' 'Scrypt' 'Scrypt' 'Scrypt' 'X11' 'Scrypt' 'Groestl' 'Multiple'
 'SHA-256' 'Scrypt' 'Scrypt' 'Scrypt' 'Scrypt' 'PoS' 'Scrypt' 'Scrypt'
 'NeoScrypt' 'Scrypt' 'Scrypt' 'Scrypt' 'Scrypt' 'X11' 'Scrypt' 'X11'
 'SHA-256' 'Scrypt' 'Scrypt' 'Scrypt' 'SHA3' 'Scrypt'
 'HybridScryptHash256' 'Scrypt' 'Scrypt' 'SHA-256' 'Scrypt' 'X13' 'Scrypt'
 'SHA-256' 'Scrypt' 'X13' 'NeoScrypt' 'Scrypt' 'Scrypt' 'Scrypt' 'Scrypt'
 'Scrypt' 'Scrypt' 'X11' 'X11' 'SHA-256' 'Multiple' 'SHA-256' 'PHI1612'
 'X11' 'SHA-256' 'SHA-256' 'SHA-256' 'X11' 'Scrypt' 'Scrypt' 'Scrypt'
 'Scrypt' 'Lyra2REv2' 'Scrypt' 'X11' 'Multiple' 'SHA-256' 'X13' 'Scrypt'
 'CryptoNight' 'CryptoNight' 'Shabal256' 'Counterparty' 'Scrypt' 'SHA-256'
 'Groestl' 'Scrypt' '

In [76]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df, x="PC 1", y="PC 2", z="PC 3",
    color="Class",
    symbol="Class",
    hover_name="CoinName",
    hover_data=["Algorithm"]
)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [77]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(
    columns=['CoinName', 'Algorithm', 'ProofType','TotalCoinSupply', 'TotalCoinsMined', 'Class'],
    sortable=True,
    selectable=True
)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [78]:
# Print the total number of tradable cryptocurrencies.
clustered_df['CoinName'].count()

532

In [79]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled_coin_supply = MinMaxScaler((0,1)).fit_transform(clustered_df[['TotalCoinSupply']])

scaled_coins_mined = MinMaxScaler((0,1)).fit_transform(clustered_df[['TotalCoinsMined']])

In [80]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
coin_supply_df = pd.DataFrame(data=scaled_coin_supply, columns=['TotalCoinSupply'], index=clustered_df.index.copy())
coins_mined_df = pd.DataFrame(data=scaled_coins_mined, columns=['TotalCoinsMined'], index=clustered_df.index.copy())
plot_df = coin_supply_df.join(coins_mined_df)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = clustered_df['CoinName']
plot_df.head(10)

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = clustered_df['Class']

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,2
ETH,0.000000e+00,0.000109,Ethereum,2
LTC,8.400000e-05,0.000064,Litecoin,2
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,2
ETC,2.100000e-04,0.000115,Ethereum Classic,2
ZEC,2.100000e-05,0.000007,ZCash,2


In [81]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols="CoinName")

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)